# C51 Distributional DQN

**C51** este primul algoritm major de *distributional reinforcement learning*.  
În loc să prezică un singur scalar Q(s, a), C51 prezice **o distribuție întreagă** peste valorile posibile ale Q.

---

## 🎯 Ideea-cheie

DQN clasic învață:

$$
Q(s,a)
$$

C51 învață:

$$
P(Z \mid s,a)
$$

unde \(Z\) este o variabilă aleatoare ce reprezintă distribuția recompenselor viitoare.  
Astfel, agentul are acces nu doar la valoare, ci și la **incertitudine**, **risc**, **variație** etc.

---

## 🎛 Suportul distribuției

C51 folosește o distribuție discretizată pe **51 de atomi** (de unde vine „C51”):

$$
z_i = v_{\min} + i \cdot \Delta z,
\qquad i = 0,1,\dots,50
$$

unde:

$$
\Delta z = \frac{v_{\max} - v_{\min}}{N_{\text{atoms}} - 1}
$$

Rețeaua prezice o distribuție softmax peste acești atomi pentru fiecare acțiune.

---

## 🔁 Projection Step (Bellman Projection)

Predicția target trebuie „proiectată” înapoi pe suportul fix:

$$
Tz = r + \gamma (1 - d) z
$$

Se calculează poziția fiecărui \(Tz_i\) între atomi și se redistribuie masa probabilistică:

- se determină pozițiile fracționate:  
  $$b = \frac{Tz - v_{\min}}{\Delta z}$$
- se proiectează înapoi prin *interpolare liniară* pe atomi vecini.

Acesta este pasul esențial care face C51 diferit de DQN.

---

## 🎮 Alegerea acțiunii

Q-value se obține ca media distribuției:

$$
Q(s,a) = \sum_i p_i \, z_i
$$

Apoi agentul alege acțiunea cu Q cel mai mare.

---

## ⭐ Avantaje

- surprinde **incertitudinea** și **riscul**  
- învățare mai stabilă decât DQN clasic  
- fundament pentru **Rainbow DQN**  
- convergență mai rapidă în multe environment-uri  
- precizie mai bună în probleme stocastice  

---

## 📌 Rezumat

**C51 = DQN + distribuție pe atomi + proiecție Bellman.**

Este primul algoritm complet distributional, mult mai expresiv decât DQN clasic și un ingredient esențial în algoritmii moderni de RL.


In [ ]:
import gymnasium as gym
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tensorflow.keras import Model, layers, optimizers


# ============================================================
# Replay Buffer
# ============================================================

class ReplayBuffer:
    def __init__(self, capacity, state_dim):
        self.capacity = capacity
        self.ptr = 0
        self.full = False

        self.states = np.zeros((capacity, state_dim), dtype=np.float32)
        self.next_states = np.zeros((capacity, state_dim), dtype=np.float32)
        self.actions = np.zeros(capacity, dtype=np.int32)
        self.rewards = np.zeros(capacity, dtype=np.float32)
        self.dones = np.zeros(capacity, dtype=np.float32)

    def store(self, s, s2, a, r, d):
        self.states[self.ptr] = s
        self.next_states[self.ptr] = s2
        self.actions[self.ptr] = a
        self.rewards[self.ptr] = r
        self.dones[self.ptr] = d

        self.ptr = (self.ptr + 1) % self.capacity
        if self.ptr == 0:
            self.full = True

    def sample(self, batch_size):
        max_size = self.capacity if self.full else self.ptr
        idx = np.random.choice(max_size, batch_size, replace=False)
        return (
            self.states[idx],
            self.next_states[idx],
            self.actions[idx],
            self.rewards[idx],
            self.dones[idx]
        )


# ============================================================
# C51 Network (Distributional Q-learning)
# ============================================================

class C51Network(Model):
    def __init__(self, n_actions, n_atoms, v_min, v_max):
        super().__init__()
        self.n_actions = n_actions
        self.n_atoms = n_atoms
        self.v_min = v_min
        self.v_max = v_max

        # Support (z-values / atoms)
        self.support = tf.linspace(v_min, v_max, n_atoms)
        self.support = tf.cast(self.support, tf.float32) 

        # Architecture
        self.fc1 = layers.Dense(128, activation='relu')
        self.fc2 = layers.Dense(128, activation='relu')
        self.logits = layers.Dense(n_actions * n_atoms)

    def call(self, x):
        x = tf.convert_to_tensor(x, dtype=tf.float32)
        x = tf.nn.relu(self.fc1(x))
        x = tf.nn.relu(self.fc2(x))

        logits = self.logits(x)
        logits = tf.reshape(logits, (-1, self.n_actions, self.n_atoms))

        probs = tf.nn.softmax(logits, axis=2)
        return probs

    def act(self, state):
        state = state[np.newaxis, :]
        dist = self.call(state)
        q_values = tf.reduce_sum(dist * self.support, axis=2) 
        return int(tf.argmax(q_values[0]).numpy())



# ============================================================
# Hyperparameters
# ============================================================

GAMMA = 0.99
LR = 1e-3
BATCH = 64
MEM_SIZE = 50000
NUM_EPISODES = 600

# C51 atoms
N_ATOMS = 51
V_MIN = -10
V_MAX = 10

# Epsilon-greedy
INITIAL_EPS = 1.0
FINAL_EPS = 0.02
EPS_DECAY = 0.995

TAU = 0.005


# ============================================================
# Soft update (Polyak averaging)
# ============================================================

def soft_update(target, online, tau=TAU):
    tw = target.get_weights()
    ow = online.get_weights()
    new_w = [t * (1 - tau) + o * tau for t, o in zip(tw, ow)]
    target.set_weights(new_w)


# ============================================================
# C51 Projection Step (core of the algorithm)
# ============================================================

def project_distribution(next_dist, rewards, dones, gamma, support, v_min, v_max):
    batch_size = rewards.shape[0]
    n_atoms = support.shape[0]
    delta_z = (v_max - v_min) / (n_atoms - 1)

    projected = np.zeros((batch_size, n_atoms), dtype=np.float32)

    # Bellman update on support
    t_z = rewards[:, None] + gamma * (1 - dones[:, None]) * support[None, :]
    t_z = np.clip(t_z, v_min, v_max)

    # Positions inside support
    b = (t_z - v_min) / delta_z
    l = np.floor(b).astype(np.int32)
    u = np.ceil(b).astype(np.int32)

    next_dist = next_dist.numpy()

    # Distribute probability mass
    for i in range(batch_size):
        for j in range(n_atoms):
            lj = l[i, j]
            uj = u[i, j]
            p = next_dist[i, j]

            if lj == uj:
                projected[i, lj] += p
            else:
                projected[i, lj] += p * (uj - b[i, j])
                projected[i, uj] += p * (b[i, j] - lj)

    return projected


# ============================================================
# Training Loop — C51 Distributional DQN
# ============================================================

env = gym.make("CartPole-v1")
state_dim = env.observation_space.shape[0]
n_actions = env.action_space.n

buffer = ReplayBuffer(MEM_SIZE, state_dim)
online = C51Network(n_actions, N_ATOMS, V_MIN, V_MAX)
target = C51Network(n_actions, N_ATOMS, V_MIN, V_MAX)
target.set_weights(online.get_weights())

optimizer = optimizers.Adam(LR)
epsilon = INITIAL_EPS
reward_history = []

for episode in range(NUM_EPISODES):

    state, _ = env.reset()
    ep_reward = 0

    for step in range(200):

        # Epsilon-greedy
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = online.act(state)

        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        ep_reward += reward

        buffer.store(state, next_state, action, reward, float(done))
        state = next_state

        # ----------------------------------------------------
        # TRAINING
        # ----------------------------------------------------
        if buffer.ptr > 1000 or buffer.full:

            states, next_states, actions, rewards, dones = buffer.sample(BATCH)

            # Next distributions
            next_dists = target(next_states)

            # Double DQN — choose action via ONLINE network
            next_q = tf.reduce_sum(next_dists * online.support, axis=2)
            next_actions = tf.argmax(next_q, axis=1, output_type=tf.int32)

            # Extract chosen distributions
            idx = tf.stack([tf.range(BATCH, dtype=tf.int32), next_actions], axis=1)
            best_dists = tf.gather_nd(next_dists, idx)

            # Projected target distribution
            projected = project_distribution(
                best_dists, rewards, dones,
                GAMMA, online.support.numpy(), V_MIN, V_MAX
            )

            # Train
            with tf.GradientTape() as tape:
                dists = online(states)
                a_idx = tf.stack([tf.range(BATCH, dtype=tf.int32), actions], axis=1)
                chosen = tf.gather_nd(dists, a_idx)

                loss = tf.nn.softmax_cross_entropy_with_logits(
                    labels=projected,
                    logits=tf.math.log(chosen + 1e-8)
                )
                loss = tf.reduce_mean(loss)

            grads = tape.gradient(loss, online.trainable_variables)
            grads = [tf.clip_by_norm(g, 5.0) for g in grads]
            optimizer.apply_gradients(zip(grads, online.trainable_variables))

            soft_update(target, online)

        if done:
            break

    epsilon = max(FINAL_EPS, epsilon * EPS_DECAY)
    reward_history.append(ep_reward)

    # ----------------------------------------------------
    # LIVE PLOT
    # ----------------------------------------------------
    if episode % 5 == 0:
        clear_output(wait=True)
        plt.figure(figsize=(10,4))

        # raw rewards
        plt.plot(
            reward_history,
            label="Reward",
            color="blue",
            alpha=0.3,
            linewidth=1
        )

        # moving average
        if len(reward_history) > 20:
            ma = np.convolve(reward_history, np.ones(20)/20, mode='valid')
            plt.plot(
                range(19, len(reward_history)),
                ma,
                label="Moving Avg (20 eps)",
                color="orange",
                linewidth=2.5
            )

        plt.title("C51 Distributional DQN — Training Progress")
        plt.xlabel("Episode")
        plt.ylabel("Reward")
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()
